# Custom Climate Profiles Generation

Many energy system modeling and other planning processes are designed to intake hourly information in order to capture short term fluctuations in conditions that affect energy supply and demand. An *Annualized Hourly Climate Profile* is a dataset that contains hourly weather conditions at a given location for an entire year, and is commonly referred to as an **8760 Climate Profile**. Climate profiles are able to represent future weather conditions due to a changing climate and can thus better inform design and planning processes for a wide range of future needs. Guidance, methodologies, and recommendations for Cal-Adapt Climate Profiles is coming soon to the <span style="color: blue;\">[AE Methods](https://analytics.cal-adapt.org/analytics/methods/)</span> for reference. A comprehensive set of pre-generated Climate Profiles for the most common use-cases is available via the <span style="color: blue;\">[Analytics Engine Additional Datasets](https://analytics.cal-adapt.org/data/more_data/)</span>.

On the Cal-Adapt: Analytics Engine, we currently provide two kinds of Climate Profiles: 
1. **Standard Year 8760**:  One year of hourly data that represents any desired statistical percentile of weather conditions for a location over a 30-year climatological period. A Standard Year builds a climate profile for each climate model on a single variable and can be used to evaluate both median and extreme conditions by selecting appropriate percentiles. Both historical and future Standard Year 8760s are available.

2. **Typical Meteorological Year 8760 (TMY)**: One year of hourly data that represents the median weather conditions for a location over a climatological period. A TMY is built from ten specific weather variables that are weighted in compliance with <span style="color: blue;\">[TMY standard methodology](https://nsrdb.nrel.gov/data-sets/tmy)</span>. TMYs statistically assess the median conditions from 30 years of model data and select the most “typical” month for each month during a year. These are then compiled into a single climate profile. The resulting TMY file includes multiple variables and has very specific formatting requirements. Both historical and future TMY (FTMY) 8760s are available.

**Intended Application**: As a user, I want to <span style="color:#FF0000">**generate a custom climate profile**</span> for my area:
1. Standard Year 8760 profile, with my variable and extreme conditions of choice.
2. TMY 8760 profile, for the closest weather station to my area. 

**Runtime**: A Standard Year 8760 can take between **5-60 minutes** to generate, depending on selections. A TMY 8760 profile will take approximately **45 minutes** to generate. Modifications to selections may increase the runtime. 

#### Step 0: Set-Up
Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [ ]:
from climakitae.explore.standard_year_profile import get_climate_profile, export_profile_to_csv
from climakitae.explore.typical_meteorological_year import TMY
from climakitae.core.data_interface import get_subsetting_options

import warnings
warnings.filterwarnings("ignore")

### Generate a Custom Standard Year Profile 

AE makes it easy to generate a custom Standard Year climate profile for you. Options for customization include: 
- **Variable**. Want to know what variables (and their associated units) are available to choose from? Check out our variable list <span style="color: blue;\">[here](https://github.com/cal-adapt/climakitae/blob/main/climakitae/data/variable_descriptions.csv)</span>.
- **Quantile**. Valid options range from 0-1.
- **Global Warming Level**. The default GWL is 1.2°C, and common GWLs include 1.2, 1.5, 2.0, 2.5, 3.0. You can also do a fully custom GWL, such as 2.3°C. The valid range of options is 0.5 - 4.0°C. 
- **Delta Signal**. Option to take a delta signal difference between your selected GWL and the historical reference GWL 1.2°C. If you want the absolute values (no delta), set "no_delta" to True. The default behavior is False. 
- **Location**. You can choose between: 
    1. A HadISD weather station, such as one in the <span style="color: blue;\">[available station list](https://github.com/cal-adapt/climakitae/blob/main/climakitae/data/hadisd_stations.csv)</span>, 
    2. A custom point location of interest, via a lat-lon coordinate pair, 
    3. A cached area, such as a county, demand forecast zone, or service territory. 
    
For the full list of cached areas available on AE, the `get_subsetting_options` function can help. Check out the <span style="color: blue;\">[basic_data_access notebook](https://github.com/cal-adapt/cae-notebooks/blob/main/data-access/basic_data_access.ipynb)</span> for how to use this function. 

In [ ]:
# Make your selections -- Select your variable, quantile, and GWL!
variable = "Air Temperature at 2m"
units = "degF"
qtile = 0.5
warming_level = 1.5
no_delta = True

# Define your location -- Uncomment your desired location option! 
# HadISD weather station: provide the station name
station_name = "Sacramento Executive Airport (KSAC)"

# Custom point location: provide a latitude and longitude
# latitude = 32.7136
# longitude = -117.2031

# Cached gridded area: provide the area name
# area_name = "Los Angeles County"

In [ ]:
# Set up the Standard Year generator
profile_selections = {
    "variable": variable,
    "resolution": "3 km",
    "q": qtile,
    "warming_level": [warming_level],
    "units": units,
    "no_delta": no_delta,

    # Location options -- uncomment based on your desired location type
    "stations": [station_name],  # uncomment for a weather station
    # "latitude": (latitude - 0.02, latitude + 0.02),  # uncomment for a using a custom coordinate location
    # "longitude": (longitude - 0.02, longitude + 0.02),  # uncomment for a custom coordinate location
    # "cached_area": area_name, # uncomment for a cached area
}

# Generate the climate profile
profile = get_climate_profile(**profile_selections)

**Export**. The following cell will export your custom generated Standard Year profile!

In [ ]:
# the function uses the previously defined profile selections to generate the output file name
export_profile_to_csv(profile, **profile_selections)


### Generate a Typical Meteorological Year Profile (TMY)

AE makes it easy to generate a custom Typical Meteorological Year climate profile for you. You can generate both a historical TMY and a future TMY (FTMY). Options for customization include: 
- **Location**. You can choose between: 
    1. A HadISD weather station, such as one in the <span style="color: blue;\">[available station list](https://github.com/cal-adapt/climakitae/blob/main/climakitae/data/hadisd_stations.csv)</span>, 
    2. A custom point location of interest, via a lat-lon coordinate pair.
- **Approach**. AE can generate a TMY profile using a **Global Warming Level** or a **Time-based Approach**: 
    - **Global Warming Level**. The default GWL is 1.2°C, and common GWLs include 1.2, 1.5, 2.0, 2.5, 3.0. You can also do a fully custom GWL, such as 2.3°C. The valid range of options is 0.5 - 4.0°C. 
    - **Time-based**. We recommend a 30-year period for TMY, such as 1990-2020. 

Note: The TMY methodology here mirrors that of the NSRDB TMY3 methodology which heavily weights the solar radiation input data. Be aware that the final selection of "typical" months may not be typical for other variables. Because a TMY represents average rather than extreme conditions, an TMY profile is not suited for designing systems to meet the worst-case conditions occurring at a location. Extreme TMY profiles (XMY) are coming to AE soon. 

In [ ]:
# Define your location -- Uncomment your desired location option!
# HadISD weather station: provide the station name
station_name = "Sacramento Executive Airport (KSAC)"

# Custom point location: provide a latitude and longitude
# latitude = 32.7136
# longitude = -117.2031

# Define your approach -- Uncomment your desired option!
# GWL based
warming_level = 1.5

# Time-based
# start_year = 1990
# end_year = 2020

In [ ]:
# Set up the TMY profile generator! The verbose option will output progress of the TMY generation
tmy = TMY(
    # Location -- uncomment your desired option
    station_name = station_name,
    # latitude = latitude,
    # longitude = longitude,

    # Approach -- uncomment your desired option
    warming_level = warming_level, 
    # start_year = start_year,
    # end_year = end_year,
    verbose=True
)

# Generate the profile!
tmy.generate_tmy()

**Export to non-EPW format.** TMY profiles are exported in .epw format by default, but can be exported as both `.csv` and `.tmy` file formats using the method `export_tmy_data` with the argument `extension="csv"` as shown below.

In [ ]:
tmy.export_tmy_data(extension="csv")